<a href="https://colab.research.google.com/github/Kiron-Ang/DFHL/blob/main/cortisol_grief.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This IPYNB file was created with Google Colab by Kiron Ang
# Print Python version and install/update/import libraries!
!python -V

!pip install -U scikit-learn > output.txt
import sklearn
print("scikit-learn", sklearn.__version__)

!pip install -U polars > output.txt
import polars
print("polars", polars.__version__)

!pip install -U matplotlib > output.txt
import matplotlib
print("matplotlib", matplotlib.__version__)

!pip install -U seaborn > output.txt
import seaborn
print("seaborn", seaborn.__version__)

!pip install -U statsmodels > output.txt
import statsmodels
print("statsmodels", statsmodels.__version__)

Python 3.10.12
scikit-learn 1.5.2
polars 1.9.0
matplotlib 3.7.1
seaborn 0.13.2
statsmodels 0.14.4


In [2]:
# read_csv() to store data in a Polars DataFrame
cortisol = polars.read_csv("cortisol - grief.csv")
cortisol = cortisol.drop_nulls()
print("cortisol", cortisol.shape)
print("cortisol contains", cortisol.columns)

cortisol (1493, 14)
cortisol contains ['record_id', 'person_id', 'name', 'cortisol', 'month', 'day', 'year', 'collection', 'awakening', 'collection_minutes', 'awakening_minutes', 'difference_minutes', 'age', 'gender']


In [ ]:
# Create three Polars DataFrames to store data for three families
family_1 = cortisol.filter(polars.col("person_id").is_in(
    [151756224, 312288096, 710554240, 410631648, 437663904]))[
    "cortisol", "month", "collection_minutes", "awakening_minutes", "difference_minutes", "age", "gender"]
print(family_1)

family_2 = cortisol.filter(polars.col("person_id").is_in(
    [980003264, 533873088, 106369672, 676175872, 575183808]))[
    "cortisol", "month", "collection_minutes", "awakening_minutes", "difference_minutes", "age", "gender"]
print(family_2)

family_3 = cortisol.filter(polars.col("person_id").is_in(
    [412766816, 712730496, 207561136, 633178880]))[
    "cortisol", "month", "collection_minutes", "awakening_minutes", "difference_minutes", "age", "gender"]
print(family_3)

shape: (452, 7)
┌──────────┬───────┬────────────────────┬───────────────────┬────────────────────┬─────┬────────┐
│ cortisol ┆ month ┆ collection_minutes ┆ awakening_minutes ┆ difference_minutes ┆ age ┆ gender │
│ ---      ┆ ---   ┆ ---                ┆ ---               ┆ ---                ┆ --- ┆ ---    │
│ f64      ┆ i64   ┆ i64                ┆ i64               ┆ i64                ┆ i64 ┆ i64    │
╞══════════╪═══════╪════════════════════╪═══════════════════╪════════════════════╪═════╪════════╡
│ 0.276    ┆ 7     ┆ 465                ┆ 300               ┆ 165                ┆ 10  ┆ 0      │
│ 0.38     ┆ 7     ┆ 415                ┆ 330               ┆ 85                 ┆ 7   ┆ 0      │
│ 0.1615   ┆ 7     ┆ 975                ┆ 330               ┆ 645                ┆ 7   ┆ 0      │
│ 0.117    ┆ 8     ┆ 490                ┆ 330               ┆ 160                ┆ 5   ┆ 0      │
│ 0.117    ┆ 8     ┆ 990                ┆ 330               ┆ 660                ┆ 5   ┆ 0      │
│ … 

In [12]:
# Create time series plots for family 1 with matplotlib.pyplot and seaborn
import matplotlib.pyplot

seaborn.set_theme(style="whitegrid")

fig, axes = matplotlib.pyplot.subplots(5, 1, figsize = (20, 20), dpi = 600)

In [ ]:
# Save the scatter plot!
scatterplot.figure.savefig("cortisol_depression_kiron.png", dpi=600, bbox_inches='tight')